## Protein-protein interaction graphs

### Getting started

Create a "R \[conda env:r_3.6\]" notebook in the "my_notebooks/week14" folder. Name this notebook "ppi_SARSCov2".

At any time if you want to stop, remember to "Save and Checkpoint" your notebook before doing "Close and Halt".

### Quick introduction to R data frame

The data frame is a special data type used to store dataset tables. Think of rows as cases, columns as variables. Each column is a vector.

Creating a dataframe - see how we are defining each column as a vector.

In [1]:
dfr1 = data.frame(ID=1:4,
                  FirstName=c("John","Jim","Jane","Jill"),
                  Beverage=c("Coffee","Tea","Tea","Coffee"),
                  Age=c(22,33,44,55) )

In [2]:
dfr1

ID,FirstName,Beverage,Age
1,John,Coffee,22
2,Jim,Tea,33
3,Jane,Tea,44
4,Jill,Coffee,55


What's the dimension of this data frame?  The first number is the number of rows, and the second number is the number of columns.

In [3]:
dim(dfr1)

[1] 4 4

There are many ways to get data out of a data frame.

In [4]:
dfr1[1,]   # First row, all columns

ID,FirstName,Beverage,Age
1,John,Coffee,22


In [5]:
dfr1[,1]   # First column, all rows

[1] 1 2 3 4

In [6]:
dfr1$Age   # Age column, all rows, 1st way
dfr1[,'Age'] # Age column, all rows, 2nd way

[1] 22 33 44 55

[1] 22 33 44 55

In [7]:
dfr1[1:2,3:4] # Rows 1 and 2, columns 3 and 4 - the beverage and age of John & Jim

Beverage,Age
Coffee,22
Tea,33


In [8]:
dfr1[c(1,3),] # Rows 1 and 3, all columns

,ID,FirstName,Beverage,Age
1,1,John,Coffee,22
3,3,Jane,Tea,44


We can get the row and column names.

In [9]:
colnames(dfr1)

[1] "ID"        "FirstName" "Beverage"  "Age"

In [10]:
rownames(dfr1)

[1] "1" "2" "3" "4"

We can also set the row or column names.

In [11]:
rownames(dfr1) = c("Person1","Person2","Person3","Person4")

In [12]:
dfr1

,ID,FirstName,Beverage,Age
Person1,1,John,Coffee,22
Person2,2,Jim,Tea,33
Person3,3,Jane,Tea,44
Person4,4,Jill,Coffee,55


We can *subset* the data frame based on certain criteria.  Who likes coffee?  Note the equality sign *==*.

In [13]:
dfr1[dfr1$Beverage=='Coffee',]

,ID,FirstName,Beverage,Age
Person1,1,John,Coffee,22
Person4,4,Jill,Coffee,55


## Reading a network into R

Upload the file "Gordon_Nat2020_SuppTable2.txt" to your "my_notebooks/week14" folder.  This file contains the experimentally determined interactions between the proteins in the SARS-Cov2 genome and the proteins in human cells (*an interactome*; reference Gordon et al., Nature 2020 https://www.nature.com/articles/s41586-020-2286-9). Each line in this file represents an interaction between a SARS-Cov2 viral protein and a human protein. 

We will first read this file into R as a data frame, then convert it to a network object.

In [14]:
net_df = read.table('Gordon_Nat2020_SuppTable2.txt',
                    sep='\t',
                    header=TRUE,
                    comment.char="",
                    quote="",
                    stringsAsFactors=FALSE)

How many interactions are there?

In [15]:
dim(net_df)

[1] 332  12

Let's take a look at the first three interactions.

In [16]:
net_df[1:3,]

Bait,Preys,PreyGene,MIST,Saint_BFDR,AvgSpec,FoldChange,Uniprot.Protein.ID,Uniprot.Protein.Description,Uniprot.Function,Structures..PDB.,Uniprot.Function.in.Disease
SARS-CoV2 E,O00203,AP3B1,0.9635501,0,4.67,46.67,AP3B1_HUMAN,AP-3 complex subunit beta-1 (Adaptor protein complex AP-3 subunit beta-1) (Adaptor-related protein complex 3 subunit beta-1) (Beta-3A-adaptin) (Clathrin assembly protein complex 3 beta-1 large chain),""" Subunit of non-clathrin- and clathrin-associated adaptor protein complex 3 (AP-3) that plays a role in protein sorting in the late-Golgi/trans-Golgi network (TGN) and/or endosomes. The AP complexes mediate both the recruitment of clathrin to membranes and the recognition of sorting signals within the cytosolic tails of transmembrane cargo molecules. AP-3 appears to be involved in the sorting of a subset of transmembrane proteins targeted to lysosomes and lysosome-related organelles. In concert with the BLOC-1 complex, AP-3 is required to target cargos into vesicles assembled at cell bodies for delivery into neurites and nerve terminals.""",,""" Hermansky-Pudlak syndrome 2 (HPS2) [MIM:608233]: A form of Hermansky-Pudlak syndrome, a genetically heterogeneous autosomal recessive disorder characterized by oculocutaneous albinism, bleeding due to platelet storage pool deficiency, and lysosomal storage defects. This syndrome results from defects of diverse cytoplasmic organelles including melanosomes, platelet dense granules and lysosomes. Ceroid storage in the lungs is associated with pulmonary fibrosis, a common cause of premature death in individuals with HPS. HPS2 differs from the other forms of HPS in that it includes immunodeficiency in its phenotype and patients with HPS2 have an increased susceptibility to infections. {ECO:0000269|PubMed:10024875}. Note=The disease is caused by mutations affecting the gene represented in this entry."""
SARS-CoV2 E,O60885,BRD4,0.9784884,0,2.67,26.67,BRD4_HUMAN,Bromodomain-containing protein 4 (Protein HUNK1),""" Chromatin reader protein that recognizes and binds acetylated histones and plays a key role in transmission of epigenetic memory across cell divisions and transcription regulation. Remains associated with acetylated chromatin throughout the entire cell cycle and provides epigenetic memory for postmitotic G1 gene transcription by preserving acetylated chromatin status and maintaining high-order chromatin structure (PubMed:23589332, PubMed:23317504, PubMed:22334664). During interphase, plays a key role in regulating the transcription of signal-inducible genes by associating with the P-TEFb complex and recruiting it to promoters. Also recruits P-TEFb complex to distal enhancers, so called anti-pause enhancers in collaboration with JMJD6. BRD4 and JMJD6 are required to form the transcriptionally active P-TEFb complex by displacing negative regulators such as HEXIM1 and 7SKsnRNA complex from P-TEFb, thereby transforming it into an active form that can then phosphorylate the C-terminal domain (CTD) of RNA polymerase II (PubMed:23589332, PubMed:19596240, PubMed:16109377, PubMed:16109376, PubMed:24360279). Promotes phosphorylation of 'Ser-2' of the C-terminal domain (CTD) of RNA polymerase II (PubMed:23086925). According to a report, directly acts as an atypical protein kinase and mediates phosphorylation of 'Ser-2' of the C-terminal domain (CTD) of RNA polymerase II; these data however need additional evidences in vivo (PubMed:22509028). In addition to acetylated histones, also recognizes and binds acetylated RELA, leading to further recruitment of the P-TEFb complex and subsequent activation of NF-kappa-B (PubMed:19103749). Also acts as a regulator of p53/TP53-mediated transcription: following phosphorylation by CK2, recruited to p53/TP53 specific target promoters (PubMed:23317504). {ECO:0000269|PubMed:16109376, ECO:0000269|PubMed:16109377, ECO:0000269|PubMed:19103749, ECO:0000269|PubMed:19596240, ECO:0000269|PubMed:22334664, ECO:0000269|PubMed:22509028, ECO:0000269|PubM

We can see that the first column is the viral protein, the second column is the UniProt ID of the human protein, and the third column is the gene symbol of the human protein. The rest of the columns provide more information about the experimental result and information about the human protein.

The `graph_from_data_frame` function in the `igraph` package can convert a data frame into a graph/network object, provided that the first two columns are the nodes in each interaction.

In [17]:
library(igraph)
net = graph_from_data_frame(d=net_df,directed=FALSE)

ERROR: Error in graph_from_data_frame(d = net_df, directed = FALSE): could not find function "graph_from_data_frame"


The `net` variable is now a graph/network object. We can take a quick look at what's inside, and also get the edges and vertices (nodes) in the network.

In [ ]:
net

In [ ]:
E(net)       # The edges of the "net" object

In [ ]:
V(net)

We can get the *degree* on each node, and plot the degree distribution.

In [ ]:
deg = degree(net)
deg

In [ ]:
hist(deg)

The `net` object can be plotted directly.

In [ ]:
plot(net)

There are many options to change how your network could look like.

In [ ]:
plot(net, vertex.label.color="black", vertex.label.cex=.5,vertex.size=2,vertex.label.dist=1)

## Finding functions of human proteins interacting with viral proteins

Gordon et al. found that human proteins that interact with different viral proteins are enriched for specific biological processes.  We will try to reproduce their result for interactors of the viral protein orf8.

First we want to get a *subnetwork* of all interactions that include orf8.

In [ ]:
orf8_sub = net_df[net_df$Bait=='SARS-CoV2 orf8',]

In [ ]:
orf8_sub[1:3,]

How many interactions are there?

In [ ]:
dim(orf8_sub)

Note that `orf8_sub` is a data frame, so we can again turn it into a graph/network object by the `graph_from_data_frame` function.

In [ ]:
orf8_subnet = graph_from_data_frame(d=orf8_sub, directed=FALSE) 

In [ ]:
plot(orf8_subnet)

The `PreyGene` column contains the gene symbols of the interacting proteins.

In [ ]:
orf8_sub$PreyGene

We now load the `clusterProfiler` package that provides functions to compute GO term enrichment and the `org.Hs.eg.db` that provides the GO annotations of human genes.

In [ ]:
library(clusterProfiler)
library(org.Hs.eg.db)

The `enrichedGO` function takes a list of gene symbols, compared them to the annotated gene functions in the human genome, and return a list of enriched GO terms for the genes in your list. Recall from last lecture that we need to do FDR correction since we are doing thousands of statistical tests. `BH` is a FDR correction method.

In [ ]:
ego <- enrichGO(orf8_sub$PreyGene, 
                OrgDb=org.Hs.eg.db, 
                keyType="SYMBOL", 
                ont="BP", 
                pvalueCutoff=0.05, 
                pAdjustMethod="BH", 
                qvalueCutoff=0.05, 
                readable=FALSE)

Let's take a look at the top 10 enriched GO terms.  How does this correspond to the results in Figure 3 and Extended Data Fig 3 of the Gordon et al paper?

In [ ]:
ego[1:10,]